In [18]:
import os
from dotenv import load_dotenv
from numpy import vectorize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import string

s=""
sample_files = []
load_dotenv()

Space_Cleared_Exts = os.environ["EXTENSIONS"].replace(" ", "")
# ------------------------------------------------------------------
# Parcourt tout les fichiers finissant avec les extensions dans .env
# ------------------------------------------------------------------
for ext in os.environ["EXTENSIONS"] :
    if(ext==",") :
        ext="["
        for doc in os.listdir("./FilesToTest/") :
            if doc.endswith(str(s)) :
                sample_files.append(''.join(doc))
        s = ""
    if(ext!="[" and ext!="]") :
        s+=ext
for doc in os.listdir("./FilesToTest/") :
    if doc.endswith(str(s.replace(" ",""))) :
        sample_files.append(''.join(doc))
 
sample_contents = [open("./FilesToTest/"+File).read() for File in sample_files]
print(sample_contents)

['No.JS\n// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', 'Content3\n\n// Normal Commentary', 'yes\n// LE CHOCOLAT\n// SQDJQSDIQSDISDISDISJDISQOQIDQSOIDQSIDJQSIDJSIOJDISQDJIOSQDOIQSDJIQSDOQSJD0IQSDOKSDOIQSJODQSJDOQKDJs', 'HEHE']


In [19]:
Sample_ContentsBis = []
Commentary_Content = []

# ---------------------------------------------------------------------------------
# Parcourt notre texte et enleve les commentaires si le texte vient d'un fichier JS
# ---------------------------------------------------------------------------------

def PurgeJSCommentary(word) :
        ReelText = ""
        CommText = ""
        FoundComm = False
        FoundHugeComm = False
        for char in range(len(sample_contents[word])) :
            if(FoundComm == False) :
                if(char+1 < len(sample_contents[word])) :
                    if(sample_contents[word][char] == "/" and sample_contents[word][char+1] == "/") :
                        FoundComm=True
                    elif(sample_contents[word][char]=="/" and sample_contents[word][char+1]=="*") :
                        FoundHugeComm=True
            
            if(FoundComm==False and FoundHugeComm == False) :
                ReelText+=sample_contents[word][char]
            else :
                CommText+=sample_contents[word][char]
            if(FoundComm and FoundHugeComm == False) :
                if(sample_contents[word][char] == "\n") :
                    FoundComm = False
            if(FoundHugeComm) :
                if(sample_contents[word][char]=="*" and sample_contents[word][char]=="\\") :
                    FoundHugeComm = False
        Commentary_Content.append(CommText)
        Sample_ContentsBis.append(ReelText)
        
for i in range(len(sample_files)) :
    if(sample_files[i].endswith(".js")) :
        PurgeJSCommentary(i)

In [20]:
print(Sample_ContentsBis)
print(Commentary_Content)

['No.JS\n', 'Content3\n\n', 'yes\n']
['// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', '// Normal Commentary', '// LE CHOCOLAT\n// SQDJQSDIQSDISDISDISJDISQOQIDQSOIDQSIDJQSIDJSIOJDISQDJIOSQDOIQSDJIQSDOQSJD0IQSDOKSDOIQSJODQSJDOQKDJs']


In [21]:
def tokenize() :
    return ""
tokenize()

''

In [22]:
# vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()

x = TfidfVectorizer()
x.fit_transform(Sample_ContentsBis)
print(x.vocabulary_)
def vectorize(Text) :
    return TfidfVectorizer().fit_transform(Text).toarray()

# similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

def similarity(doc1,doc2) :
    return cosine_similarity([doc1,doc2])

{'no': 2, 'js': 1, 'content3': 0, 'yes': 3}


In [23]:

vectors = vectorize(Sample_ContentsBis)
s_vectors = list(zip(sample_files, vectors))

In [24]:

def check_plagiarism():
    results = set()
    global s_vectors
    for sample_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((sample_a, text_vector_a))
        del new_vectors[current_index]
        for sample_b, text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            sample_pair = sorted((sample_a, sample_b))
            score = sample_pair[0], sample_pair[1], sim_score
            results.add(score)
    return results

In [25]:
for data in check_plagiarism():
    print(data)

('no.js', 'testingfile.js', 0.0)
('testingfile.js', 'yes.js', 0.0)
('no.js', 'yes.js', 0.0)
